<a href="https://colab.research.google.com/github/luist-ufms/PI2_ufms/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q
!npm install -g localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 112.7 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 2s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [2]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

# ==========================================
# 1. CONFIGURAÇÃO DA PÁGINA
# ==========================================
st.set_page_config(page_title="Simulador de Fornos", layout="wide", page_icon="🏭")

# --- Barra Lateral ---
with st.sidebar:
    st.header("Configurações")

    st.subheader("1. Dados")
    uploaded_file = st.sidebar.file_uploader("Carregar 'Base_anglo.csv'", type=["csv"])

    st.divider()
    st.subheader("2. Hiperparâmetros IA")
    n_cl = st.slider("Número de Clusters", 2, 6, 3)
    max_dep = st.slider("Profundidade Árvore", 2, 20, 10)
    n_estim = st.slider("Estimadores AdaBoost", 10, 100, 30)

    if st.button("🔄 Re-treinar Modelos"):
        st.cache_resource.clear()

# ==========================================
# 2. CLASSES E FUNÇÕES (BACKEND)
# ==========================================
class Filtros:
    def nao_numerico(self, df):
        return df.select_dtypes(include=['number', 'float64', 'int64'])
    def nao_negativo(self, df):
        df = df.copy()
        num_cols = df.select_dtypes(include=['number']).columns
        df[num_cols] = df[num_cols].clip(lower=0)
        return df
    def clusterizacao(self, n_clusters, df):
        modelo = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        indices = modelo.fit_predict(df)
        return indices, modelo

def plot_comparacao_st(nome_variavel, y_real, y_pred, n_plot):
    n_plot = min(n_plot, len(y_real))
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(range(n_plot), y_real[:n_plot], 'ro--', markersize=4, label='Dados Reais')
    ax.plot(range(n_plot), y_pred[:n_plot], 'bo--', markersize=4, label='Modelo IA')
    ax.set_title(f"Validação: {nome_variavel}")
    ax.set_ylabel("Valor")
    ax.set_xlabel("Amostras de Teste")
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.5)
    return fig

@st.cache_resource
def treinar_modelo_global(df, _max_dep, _n_estim, _n_clusters, cols_in, cols_out):
    # Treina com a base completa para uso nos simuladores
    filtro = Filtros()
    indices, kmeans = filtro.clusterizacao(_n_clusters, df[cols_in])

    df_proc = df.copy()
    df_proc['K classes'] = indices

    modelos = {}
    for i in range(_n_clusters):
        df_c = df_proc[df_proc['K classes'] == i]
        if len(df_c) < 5: continue

        mods_saida = {}
        for out_col in cols_out:
            regr = AdaBoostRegressor(
                DecisionTreeRegressor(max_depth=_max_dep),
                n_estimators=_n_estim, random_state=42
            )
            regr.fit(df_c[cols_in], df_c[out_col])
            mods_saida[out_col] = regr

        modelos[i] = mods_saida

    return kmeans, modelos, indices

# ==========================================
# 3. PREPARAÇÃO DE DADOS (CARREGAMENTO)
# ==========================================
# Definição das colunas baseadas na estrutura da sua base (39 entradas + 9 saídas)
cols_in_padrao = [f"Var_Processo_{i}" for i in range(1, 40)]
cols_out_padrao = ["Temp_Zona_1", "Temp_Zona_2", "Temp_Zona_3", "Temp_Escoria",
                   "Gas_CO", "Gas_CO2", "Gas_H2", "Gas_O2", "Pressao_Interna"]

if uploaded_file is None:
    # Gerar dados aleatórios com a estrutura correta
    total_cols = len(cols_in_padrao) + len(cols_out_padrao)
    df = pd.DataFrame(np.random.rand(200, total_cols) * 100, columns=cols_in_padrao + cols_out_padrao)
    cols_in = cols_in_padrao
    cols_out = cols_out_padrao
    st.info("ℹ️ Usando dados simulados. Faça upload do CSV para dados reais.")
else:
    try:
        df = pd.read_csv(uploaded_file, sep=';', decimal=',', encoding='latin-1')
        for c in ["DateTime", "Potencia_Ativa_Total", "Temp. da Escória"]:
            if c in df.columns: df.drop(columns=c, inplace=True)

        f = Filtros()
        df = f.nao_numerico(df)
        df = f.nao_negativo(df)

        # Tenta inferir as colunas pela posição (39 primeiras são entrada, resto saída)
        cols_in = df.columns[:39].tolist()
        cols_out = df.columns[39:].tolist()

        # Se não tiver saídas suficientes na detecção, usa as últimas 9
        if len(cols_out) < 1:
             cols_out = df.columns[-9:].tolist()
             cols_in = df.columns[:-9].tolist()

        st.success(f"✅ Base Carregada: {df.shape[0]} linhas.")
    except Exception as e:
        st.error(f"Erro ao ler CSV: {e}")
        st.stop()

# Título Principal
st.title("🏭 Sistema Inteligente de Predição de Fornos")

# ==========================================
# 4. ABAS DA APLICAÇÃO
# ==========================================
tab_hist, tab_manual, tab_val = st.tabs([
    "📋 Histórico (Real vs Previsto)",
    "🎛️ Simulador Manual (Otimização)",
    "📈 Validação & Gráficos"
])

# --- TREINAMENTO GLOBAL (Executado uma vez) ---
with st.spinner("Processando inteligência..."):
    kmeans_global, modelos_global, labels_global = treinar_modelo_global(
        df, max_dep, n_estim, n_cl, cols_in, cols_out
    )

# --- ABA 1: HISTÓRICO (COMPARAÇÃO COM MAPE) ---
with tab_hist:
    st.subheader("Auditoria de Dados Históricos")
    st.markdown("Selecione uma linha do passado para comparar o Real com o Previsto pela IA.")

    c1, c2 = st.columns([1, 2])

    with c1:
        idx = st.number_input("Selecione o índice da linha:", 0, len(df)-1, 0)
        # Mostra os dados de entrada dessa linha (Read Only)
        st.caption("Parâmetros de Entrada Reais:")
        st.dataframe(df.iloc[idx][cols_in].to_frame().T, hide_index=True)

        btn_check = st.button("🔍 Calcular Erro", type="primary")

    with c2:
        if btn_check:
            # Pega entrada
            entrada_real = df.iloc[idx][cols_in].to_frame().T

            # Pega saida real
            saida_real = df.iloc[idx][cols_out].values

            # IA Prevê
            cluster = kmeans_global.predict(entrada_real)[0]
            st.info(f"Regime Operacional: **Cluster {cluster}**")

            if cluster in modelos_global:
                saida_prevista = []
                for out in cols_out:
                    val = modelos_global[cluster][out].predict(entrada_real)[0]
                    saida_prevista.append(val)

                # --- CÁLCULO DO ERRO PERCENTUAL (NOVA LÓGICA) ---
                # Evita divisão por zero adicionando um valor ínfimo se necessário
                saida_real_safe = np.array(saida_real)
                # Substitui zeros por 0.0001 para não quebrar a conta, se houver
                saida_real_safe[saida_real_safe == 0] = 0.0001

                erro_percentual = np.abs((saida_real_safe - np.array(saida_prevista)) / saida_real_safe) * 100

                # Monta Tabela Comparativa
                df_comp = pd.DataFrame({
                    "Variável": cols_out,
                    "Valor Real": saida_real,
                    "Valor Previsto (IA)": saida_prevista,
                    "Erro (%)": erro_percentual
                })

                # Formatação condicional (Destaque para erros altos)
                st.dataframe(
                    df_comp.style.format({
                        "Valor Real": "{:.2f}",
                        "Valor Previsto (IA)": "{:.2f}",
                        "Erro (%)": "{:.2f}%"
                    }).background_gradient(cmap="Reds", subset=["Erro (%)"]),
                    use_container_width=True,
                    hide_index=True
                )

                # Média do erro dessa linha
                st.metric("MAPE Médio desta linha", f"{np.mean(erro_percentual):.2f}%")

            else:
                st.warning("Cluster sem dados suficientes.")

# --- ABA 2: SIMULADOR MANUAL (PLAYGROUND) ---
with tab_manual:
    st.subheader("Simulador de Cenários (Otimização)")
    st.markdown("Altere os parâmetros de entrada abaixo para prever o comportamento do forno.")

    col_man_L, col_man_R = st.columns([1, 1])

    with col_man_L:
        st.write("**Ajuste os 39 Parâmetros de Entrada:**")
        # Inicializa com a média para facilitar
        input_medio = df[cols_in].mean().to_frame().T

        # Tabela editável
        user_input = st.data_editor(
            input_medio,
            height=500,
            use_container_width=True,
            hide_index=True,
            key="editor_manual"
        )

        btn_sim_manual = st.button("🚀 Simular Cenário", type="primary", use_container_width=True)

    with col_man_R:
        st.write("**Saídas Previstas:**")
        if btn_sim_manual:
            cluster_man = kmeans_global.predict(user_input)[0]
            st.success(f"Regime Previsto: **Cluster {cluster_man}**")

            if cluster_man in modelos_global:
                preds_man = []
                for out in cols_out:
                    val = modelos_global[cluster_man][out].predict(user_input)[0]
                    preds_man.append(val)

                # Exibe Resultados
                df_res_man = pd.DataFrame({
                    "Variável de Saída": cols_out,
                    "Previsão": preds_man
                })

                # Destaque visual
                st.dataframe(
                    df_res_man.style.format({"Previsão": "{:.2f}"}).background_gradient(cmap="Blues", subset=["Previsão"]),
                    use_container_width=True,
                    hide_index=True
                )
            else:
                st.error("Cluster fora da faixa de operação conhecida.")
        else:
            st.info("👈 Edite a tabela e clique em Simular.")

# --- ABA 3: VALIDAÇÃO (GRÁFICOS) ---
with tab_val:
    st.subheader("Análise de Acurácia (Testes)")

    c_val1, c_val2 = st.columns(2)
    with c_val1:
        var_alvo = st.selectbox("Variável para Analisar:", cols_out)
    with c_val2:
        cluster_analise = st.selectbox("Filtrar por Cluster:", sorted(list(set(labels_global))))

    if st.button("Gerar Gráfico de Validação"):
        with st.spinner("Processando divisão treino/teste..."):
            # Filtra
            df_temp = df.copy()
            df_temp['K'] = labels_global
            df_cluster = df_temp[df_temp['K'] == cluster_analise]

            if len(df_cluster) > 10:
                # Split
                X = df_cluster[cols_in]
                y = df_cluster[var_alvo]
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                # Treina modelo exclusivo para validação
                regr_val = AdaBoostRegressor(DecisionTreeRegressor(max_depth=max_dep), n_estimators=n_estim)
                regr_val.fit(X_train, y_train)
                y_pred_val = regr_val.predict(X_test)

                # Métricas
                mape = np.mean(np.abs((y_test - y_pred_val) / y_test)) * 100

                st.metric("Erro Médio (MAPE)", f"{mape:.2f}%")
                st.pyplot(plot_comparacao_st(var_alvo, y_test.values, y_pred_val, 100))
            else:
                st.error("Dados insuficientes neste cluster.")

Writing app.py


In [3]:
# --- Célula 4: Rodar com Cloudflare (Mais estável) ---
import sys

# 1. Baixa a ferramenta do Cloudflare (silenciosamente)
!wget -q -O cloudflared-linux-amd64 https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

# 2. Roda o Streamlit em background
print("Iniciando Streamlit...")
!streamlit run app.py &>/dev/null &

# 3. Cria o túnel e exibe o link
print("Gerando link público (aguarde 5 segundos)...")
import time; time.sleep(5)
!./cloudflared-linux-amd64 tunnel --url http://localhost:8501

Iniciando Streamlit...
Gerando link público (aguarde 5 segundos)...
2025-11-26T19:52:03Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-26T19:52:03Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-26T19:52:08Z INF +--------------------------------------------------------------------------------------------+
2025-11-26T19:52:08Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-26T19